In [ ]:
pip install transformers datasets

In [2]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [33]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments


In [29]:
def prepare_input(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding='max_length')

    # Move inputs to the device if using GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}
    return inputs

In [31]:
def generate_summary(text):
    # Prepare the input
    inputs = prepare_input(text)

    # Generate summary
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=150, num_beams=5, length_penalty=2.0, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
df = pd.read_csv('/content/summarizdataset.csv')

In [9]:
df.head()

,text,type,Processed Text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,culture,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


In [35]:
def clean_text(text):
    text = text.strip()
    text = ' '.join(text.split())
    return text

df['text'] = df['text'].apply(clean_text)
df['summarizer'] = df['summarizer'].apply(clean_text)

In [36]:
subset_df = df.sample(n=800, random_state=42)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(subset_df)
dataset = dataset.train_test_split(test_size=0.1)

In [11]:
# Step 2: Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

def preprocess_function(examples):
    inputs = tokenizer(examples["Processed Text"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["summarizer"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [12]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
# Step 3: Load the model and training arguments
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small").to(device)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
training_args = Seq2SeqTrainingArguments(
       output_dir="./results",
       evaluation_strategy="epoch",
       learning_rate=2e-5,
       per_device_train_batch_size=8,  # Reduced batch size
       per_device_eval_batch_size=8,  # Reduced batch size
       weight_decay=0.01,
       save_total_limit=3,
       num_train_epochs=3,
       predict_with_generate=True,
       report_to="none",
)

In [18]:
# Step 4: Define the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

In [19]:
# Step 5: Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Step 6: Evaluate the model
results = trainer.evaluate()
print(results)

In [ ]:
# Step 7: Generate summaries
def generate_summary(article):
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(device)
    summary_ids = model.generate(inputs["input_ids"], max_length=128, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
article = "أبوظبي: آلاء عبد الغني تناولت الجلسة الثانية في اليوم الثاني للقمة الثقافية أمس..."
summary = generate_summary(article)
print(summary)